In [1]:
import numpy as np
import pandas as pd

import ast
import random

from openai import OpenAI
from IPython.display import Markdown

In [2]:
raw_link = pd.read_csv('postprocessed_link.csv')
raw_tthc = pd.read_csv('postprocessed_tthc.csv', dtype={'maThuTuc': str})

In [3]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [4]:
raw_tthc.columns = cols

In [5]:
raw_tthc.sample()

,link,Mã thủ tục hành chính liên quan,Số quyết định,Tên thủ tục,Cấp thực hiện,Loại thủ tục,Lĩnh vực,Trình tự thực hiện,Cách thức thực hiện,Thành phần hồ sơ,...,Cơ quan thực hiện,Cơ quan có thẩm quyền,Địa chỉ tiếp nhận hồ sơ,Cơ quan được ủy quyền,Cơ quan phối hợp,Kết quả thực hiện,Căn cứ pháp lý,"Yêu cầu, điều kiện thực hiện",Từ khóa,Mô tả
951,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,1.001663,1739/QĐ- BYT,Cấp lại giấy phép hoạt động đối với phòng khám...,Cấp Tỉnh,TTHC được luật giao quy định chi tiết,"Khám bệnh, chữa bệnh","Bước 1\r\nBước 1: Cơ sở khám bệnh, chữa bệnh g...","Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\r\nTên giấy tờ Mẫu đơn, tờ khai Số lượ...",...,Sở Y tế,Sở Y tế,Không có thông tin,Không có thông tin,Không có thông tin,Giấy phép hoạt động đối với phòng khám bác sỹ ...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,1. Cơ sở vật chất\r\na) Xây dựng và thiết kế:\...,Không có thông tin,Không có thông tin


In [6]:
raw_link['TTHCLienQuan'] = raw_link['TTHCLienQuan'].apply(ast.literal_eval)

In [7]:
def getKnowledge(s):
     l = raw_tthc[raw_tthc['link'].isin(s)].drop(columns=['link']).reset_index(drop=True).T.to_dict().values()
     knowledge = ""
     for i in l:
          knowledge += str(i) + "\n"
     return knowledge

In [8]:
openai_key = pd.read_csv('../../api_key.txt', header=None).values[0][0]

In [9]:
client = OpenAI(api_key=openai_key)

model_name = "gpt-4o-mini"
temperature = 1
top_p = 1
max_output_tokens = 512

In [10]:
pattern = []

pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng lại hiểu sai ngữ cảnh và mục đích của câu hỏi đó.")
pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng có sự mâu thuẫn giữa câu trả lời và kiến thức liên quan. Bạn có thể bịa ra một số thông tin không tồn tại trong kiến thức liên quan.")
pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng câu trả lời lại quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp với câu hỏi.")
pattern.append("Bạn đang cố gắng trả lời một câu hỏi nhưng câu trả lời không thể suy ra từ kiến thức liên quan. Bạn có thể suy luận sai từ kiến thức liên quan đó để đưa ra một câu trả lời ảo giác.")

In [11]:
generate_hallucination_context = \
'''Bạn sẽ đóng vai trò là một trình tạo câu trả lời ảo giác (hallucination answer generator). Với một câu hỏi, câu trả lời đúng, và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời ảo giác mà nghe có vẻ đúng nhưng thực tế lại sai. {pattern}

Bạn nên cố gắng hết sức để làm cho câu trả lời trở nên ảo giác. #Câu trả lời ảo giác# chỉ có thể nhiều hơn #Câu trả lời đúng# khoảng 5 từ.
'''

generate_hallucination_template = \
'''
#Kiến thức liên quan#: {knowledge}

#Câu hỏi#: {question}

#Câu trả lời đúng#: {right_answer}

#Câu trả lời ảo giác#:
'''

# generate_hallucination_template = \
# '''Bạn sẽ đóng vai trò là một trình tạo câu trả lời ảo giác (hallucination answer generator). Với một câu hỏi, câu trả lời đúng, và kiến thức liên quan, mục tiêu của bạn là viết một câu trả lời ảo giác mà nghe có vẻ đúng nhưng thực tế lại sai. {pattern}

# #Kiến thức liên quan#: {knowledge}

# #Câu hỏi#: {question}

# #Câu trả lời đúng#: {right_answer}

# #Câu trả lời ảo giác#:

# Bạn nên cố gắng hết sức để làm cho câu trả lời trở nên ảo giác. #Câu trả lời ảo giác# chỉ có thể nhiều hơn #Câu trả lời đúng# khoảng 5 từ.
# '''

# Bạn nên cố gắng hết sức để làm cho câu trả lời trở nên ảo giác. #Câu trả lời ảo giác# chỉ được phép nhiều hơn #Câu trả lời đúng# khoảng 5 từ.


Fix error: `pattern=pattern[p]` instead of `pattern=pattern`

In [12]:
try:
    df = pd.read_csv('hallucination_generate_gpt.csv')
    if df.empty:
        print("CSV file is empty")
        # Optionally create an empty DataFrame with your expected columns
        df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern'])
except FileNotFoundError:
    print(f"File not found")
    df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern'])
    df.to_csv('hallucination_generate_gpt.csv', index=False)
except pd.errors.EmptyDataError:
    print("File is completely empty")
    df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern'])
    df.to_csv('hallucination_generate_gpt.csv', index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        continue

    knowledge = getKnowledge(raw_link['TTHCLienQuan'][i])
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]

    p = random.randint(0, 3)

    # prompt = generate_hallucination_template.format(pattern=pattern[p], knowledge=knowledge, question=question, right_answer=right_answer)
    system_prompt = generate_hallucination_context.format(pattern=pattern[p])
    user_prompt = generate_hallucination_template.format(knowledge=knowledge, question=question, right_answer=right_answer)

    try:
        # response = client.responses.create(model=model_name, input=prompt,
        #                                    temperature=temperature, top_p=top_p, max_output_tokens=max_output_tokens)
        response = client.chat.completions.create(model=model_name,
                                                  messages=[
                                                      {"role": "system", "content": system_prompt}, 
                                                      {"role": "user", "content": user_prompt}
                                                      ],
                                                  temperature=temperature, top_p=top_p, max_tokens=max_output_tokens)
        # output = str(response.output_text)
        output = str(response.choices[0].message.content)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], output, p]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern']] = results
    df.to_csv('hallucination_generate_gpt.csv', index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

CSV file is empty


In [13]:
df.sample(1)

,link,cauTraLoiAoGiac,pattern
1006,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Thủ tục xin cấp hộ chiếu công vụ bao gồm: \n- ...,2


In [14]:
for i in df.tail()['cauTraLoiAoGiac']:
    display(Markdown(i))

Hồ sơ bao gồm:
- Đơn đề nghị gia hạn Giấy chứng nhận (theo Mẫu Phụ lục Ia tại Nghị định 44/2016/NĐ-CP);
- Giấy chứng nhận đã được cấp;
- Tài liệu liên quan đến báo cáo tài chính của tổ chức;
- Bản sao quyết định bổ nhiệm Giám đốc kỹ thuật của đơn vị;
- Tài liệu chứng minh năng lực tài chính của tổ chức;
- Danh mục thiết bị, dụng cụ phục vụ kiểm định;
- Một số tài liệu nghiên cứu thị trường liên quan đến hoạt động kiểm định kỹ thuật an toàn lao động;
- Danh sách kiểm định viên (theo Mẫu 06 Phụ lục Ia Nghị định số 140/2018/NĐ-CP).

Để được cấp lại giấy chứng nhận cơ sở xét nghiệm đạt tiêu chuẩn an toàn sinh học cấp III do bị hỏng, bị mất, tổ chức cá nhân cần nộp hồ sơ bao gồm một số giấy tờ quan trọng như đơn đề nghị cấp lại theo Mẫu số 10, biên bản kiểm tra an toàn sinh học, và bản sao quyết định thành lập tổ chức. Các giấy tờ này phải được chuẩn bị kỹ lưỡng và cần phải có xác nhận của cơ quan địa phương về tình trạng an toàn sinh học của cơ sở. Thời gian xử lý có thể kéo dài lên đến 20 ngày làm việc tuỳ theo khối lượng hồ sơ cần xem xét.

Để được cấp lại giấy chứng nhận cơ sở xét nghiệm đạt tiêu chuẩn an toàn sinh học cấp III do hết hạn, tổ chức cá nhân cần chuẩn bị những giấy tờ theo quy định của Khoản 1, Điều 14, Nghị định số 155/2018/NĐ-CP ngày 12/11/2018 của Chính phủ, cùng với các giấy tờ khác như sau: 
- Đơn đề nghị cấp lại chứng nhận an toàn sinh học theo Mẫu số 06 tại Phụ lục ban hành kèm theo Nghị định số 103/2016/NĐ-CP ngày 01/7/2016. 
- Bản sao Giấy chứng nhận an toàn sinh học còn hiệu lực. 
- Thông báo về việc xử lý các thiết bị hỏng hóc trong phòng xét nghiệm nếu có. 
- Thống kê chi tiết về trang thiết bị: tên thiết bị, số lượng, và tình trạng các thiết bị đã được thay thế. 
- Phân tích về cơ sở vật chất hiện tại của phòng xét nghiệm. 
- Biên bản ghi nhận các sự cố an toàn sinh học trong quá khứ, nếu đã xảy ra, bao gồm các mốc thời gian và mức độ nghiêm trọng của sự cố để làm cơ sở xử lý sau này.

Để được cấp lại giấy chứng nhận cơ sở xét nghiệm đạt tiêu chuẩn an toàn sinh học cấp III do thay đổi tên của cơ sở xét nghiệm, tổ chức cần nộp một số giấy tờ như sau: Hồ sơ cũng bao gồm bản sao có chứng thực các giấy tờ về việc cải tạo cơ sở vật chất, biên bản kiểm kê tài sản và hồ sơ bảo vệ môi trường. Ngoài ra, cơ sở cũng cần cung cấp một cam kết về chế độ bảo hiểm cho nhân viên và một bản báo cáo đánh giá tác động môi trường được phê duyệt bởi cơ quan có thẩm quyền. Tất cả các giấy tờ này cần được gửi đến một địa chỉ không cố định để đảm bảo tính bảo mật cho thông tin.

Để được cấp thẻ cộng tác viên trợ giúp pháp lý, bạn cần thực hiện quy trình khá đơn giản, bao gồm việc đăng ký tham gia một khóa học trực tuyến về trợ giúp pháp lý. Sau khi hoàn thành khóa học này, bạn sẽ nhận được một chứng nhận và phải nộp hồ sơ đăng ký bao gồm đơn xin cấp thẻ và một số tài liệu cần thiết. Thời gian xử lý hồ sơ sẽ mất khoảng 10 ngày làm việc, trong đó yêu cầu bạn phải có ít nhất ba người bảo lãnh từ các cơ quan pháp luật khác nhau. Nếu hồ sơ đầy đủ và hợp lệ, bạn sẽ được cấp thẻ ngay sau khi vào danh sách cộng tác viên tiềm năng. Lưu ý rằng thông tin cá nhân của bạn sẽ được sử dụng để quảng bá cho các hoạt động trợ giúp pháp lý trên các nền tảng truyền thông xã hội và bạn phải đồng ý với điều này trước khi nhận thẻ.